### Imports

In [1]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import get_dataset_class
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")

DS = "OfficeHome"
MODEL_NAME = "resnet18"

Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu126 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info
W1114 19:56:00.442000 9056 torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


Device: cuda
PyTorch: 2.8.0+cu126


### DataLoading

In [2]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

DatasetClass = get_dataset_class(DS)

ld = DatasetClass(
    data_root=CFG["datasets"][DS]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [3]:
dataset_name = DS
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-11-14 19:56:04,331 | INFO | Initialized experiment directories for OfficeHome
2025-11-14 19:56:04,332 | INFO | Logs: d:\Haseeb\SPROJ\GRQO\Vit-GRQO\resnet18_experiments\OfficeHome\logs
2025-11-14 19:56:04,333 | INFO | Checkpoints: d:\Haseeb\SPROJ\GRQO\Vit-GRQO\resnet18_experiments\OfficeHome\checkpoints
2025-11-14 19:56:04,334 | INFO | Plots: d:\Haseeb\SPROJ\GRQO\Vit-GRQO\resnet18_experiments\OfficeHome\plots


### Setup

In [4]:
domains = CFG["datasets"][DS]["domains"]
loaders = {d: {"train": ld.get_dataloader(d, train=True), "val": ld.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset=DS)
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]

{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [5]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

2025-11-14 19:56:07,224 | INFO | === LODO: Leaving out domain 'Art' ===



=== LODO: Leaving out domain 'Art' ===


Evaluating: 100%|██████████| 19/19 [00:53<00:00,  2.81s/it]
2025-11-14 20:00:28,957 | INFO | [Art] Epoch 1/5 | Train - Loss: 2.4304, Cls: 2.4218, GRQO: 0.0085, Acc: 0.4690 | Val - Loss: 1.9660, Cls: 1.9634, GRQO: 0.0026, Acc: 0.5208
2025-11-14 20:00:29,066 | INFO | [Art] New best val acc: 0.5208


[Art] Epoch 1/5 | Train - Loss: 2.4304, Cls: 2.4218, GRQO: 0.0085, Acc: 0.4690 | Val - Loss: 1.9660, Cls: 1.9634, GRQO: 0.0026, Acc: 0.5208
[Art] New best val acc: 0.5208


Evaluating: 100%|██████████| 19/19 [00:11<00:00,  1.69it/s]
2025-11-14 20:02:10,885 | INFO | [Art] Epoch 2/5 | Train - Loss: 0.7439, Cls: 0.7396, GRQO: 0.0043, Acc: 0.8261 | Val - Loss: 1.7799, Cls: 1.7784, GRQO: 0.0015, Acc: 0.5550
2025-11-14 20:02:10,953 | INFO | [Art] New best val acc: 0.5550


...

[Real World] Epoch 3/5 | Train - Loss: 0.3611, Cls: 0.3576, GRQO: 0.0036, Acc: 0.9249 | Val - Loss: 1.0741, Cls: 1.0733, GRQO: 0.0008, Acc: 0.7177
[Real World] New best val acc: 0.7177


Evaluating: 100%|██████████| 35/35 [00:50<00:00,  1.43s/it]
2025-11-15 00:23:43,682 | INFO | [Real World] Epoch 4/5 | Train - Loss: 0.1449, Cls: 0.1418, GRQO: 0.0031, Acc: 0.9748 | Val - Loss: 1.0526, Cls: 1.0520, GRQO: 0.0007, Acc: 0.7333
2025-11-15 00:23:43,740 | INFO | [Real World] New best val acc: 0.7333


[Real World] Epoch 4/5 | Train - Loss: 0.1449, Cls: 0.1418, GRQO: 0.0031, Acc: 0.9748 | Val - Loss: 1.0526, Cls: 1.0520, GRQO: 0.0007, Acc: 0.7333
[Real World] New best val acc: 0.7333


Evaluating: 100%|██████████| 35/35 [00:49<00:00,  1.41s/it]
2025-11-15 00:25:07,239 | INFO | [Real World] Epoch 5/5 | Train - Loss: 0.0770, Cls: 0.0743, GRQO: 0.0027, Acc: 0.9869 | Val - Loss: 1.0565, Cls: 1.0560, GRQO: 0.0005, Acc: 0.7356
2025-11-15 00:25:07,306 | INFO | [Real World] New best val acc: 0.7356
2025-11-15 00:25:07,306 | INFO | [Real World] Best Acc: 0.7356
2025-11-15 00:25:07,306 | INFO | ------------------------------------------------------------
2025-11-15 00:25:07,306 | INFO | LODO finished | Mean Acc: 0.6252 | Summary saved to d:\Haseeb\SPROJ\GRQO\Vit-GRQO\resnet18_experiments\OfficeHome\logs\lodo_summary_20251115_002507.json


[Real World] Epoch 5/5 | Train - Loss: 0.0770, Cls: 0.0743, GRQO: 0.0027, Acc: 0.9869 | Val - Loss: 1.0565, Cls: 1.0560, GRQO: 0.0005, Acc: 0.7356
[Real World] New best val acc: 0.7356
[Real World] Best Acc: 0.7356
------------------------------------------------------------
LODO finished | Mean Acc: 0.6252
Summary saved to d:\Haseeb\SPROJ\GRQO\Vit-GRQO\resnet18_experiments\OfficeHome\logs\lodo_summary_20251115_002507.json


### Baseline

In [6]:
baseline_results, baseline_mean = run_baseline(
    model_name=MODEL_NAME,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)

2025-11-15 00:25:07,318 | INFO | Initializing ResNet baseline: resnet18
2025-11-15 00:25:07,389 | INFO | === Baseline LODO: Leaving out domain 'Art' ===


Initializing ResNet baseline: resnet18

=== Baseline LODO: Leaving out domain 'Art' ===


2025-11-15 00:26:29,902 | INFO | [Art] Epoch 1/5 | Train - Loss: 2.2091, Acc: 0.5287 | Val Acc: 0.4891


[Art] Epoch 1/5 | Train - Loss: 2.2091, Acc: 0.5287 | Val Acc: 0.4891


2025-11-15 00:27:52,161 | INFO | [Art] Epoch 2/5 | Train - Loss: 0.8353, Acc: 0.8367 | Val Acc: 0.5315


...

[Real World] Epoch 3/5 | Train - Loss: 0.5274, Acc: 0.9094 | Val Acc: 0.7475


2025-11-15 00:51:20,292 | INFO | [Real World] Epoch 4/5 | Train - Loss: 0.2626, Acc: 0.9649 | Val Acc: 0.7498


[Real World] Epoch 4/5 | Train - Loss: 0.2626, Acc: 0.9649 | Val Acc: 0.7498


2025-11-15 00:52:42,682 | INFO | [Real World] Epoch 5/5 | Train - Loss: 0.1319, Acc: 0.9844 | Val Acc: 0.7501
2025-11-15 00:52:42,682 | INFO | [Real World] Best Val Acc: 0.7501
2025-11-15 00:52:42,682 | INFO | ------------------------------------------------------------
2025-11-15 00:52:42,684 | INFO | Baseline LODO (resnet18) finished | Mean Acc: 0.6290


[Real World] Epoch 5/5 | Train - Loss: 0.1319, Acc: 0.9844 | Val Acc: 0.7501
[Real World] Best Val Acc: 0.7501
------------------------------------------------------------
Baseline LODO (resnet18) finished | Mean Acc: 0.6290
